In [1]:
from flask import Flask, jsonify, request
from flask_cors import CORS
import pandas as pd
import pymongo
from fuzzywuzzy import process
from transformers import pipeline
import os

In [2]:
app = Flask(__name__)
CORS(app)

In [3]:
# Load environment variables (only in development)
if os.environ.get("FLASK_ENV") != "production":
    from dotenv import load_dotenv
    load_dotenv()

In [4]:
# MongoDB setup
mongo_url = os.getenv('MONGO_URL')
client = pymongo.MongoClient(mongo_url)
db = client['test']
collection = db['policies']

In [5]:
# Load data into a DataFrame
data = pd.DataFrame(list(collection.find()))

In [6]:
# Initialize the question-answering pipeline
qa_pipeline = pipeline("question-answering", model="distilbert-base-cased-distilled-squad")

C:\Users\moham\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [7]:
@app.route('/')
def home():
    return 'Hello World'

In [8]:
@app.route('/qahandbook/<query>', methods=['POST'])
def get_policy_answer(query):
    df = data
    try:
        # Fuzzy search for policy
        return jsonify({'policy': search_policy_fuzzy(df, query), 'description': '  Description of the policy...'})
    except Exception as e:
        # Fallback to QA pipeline if fuzzy search fails
        answer = answer_question(df, query)
        return jsonify({'policy': query, 'description': answer})

def search_policy_fuzzy(data, query):
    best_match = process.extractOne(query, data['policy'].tolist())
    if best_match:
        return data[data['policy'] == best_match[0]]['description'].values[0]
    else:
        return "No relevant policy found."

def answer_question(data, question):
    for context in data['description'].tolist():
        result = qa_pipeline({'question': question, 'context': context})
        if result['score'] > 0.2:  # Confidence threshold
            return result['answer']
    return "No relevant policy found."

In [9]:
if __name__ == '__main__':
    app.run(debug=False, host="0.0.0.0", port=5000)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.0.16:5000
Press CTRL+C to quit
127.0.0.1 - - [16/Sep/2024 21:46:21] "OPTIONS /qahandbook/remote%20w HTTP/1.1" 200 -
127.0.0.1 - - [16/Sep/2024 21:46:21] "POST /qahandbook/remote%20w HTTP/1.1" 200 -
127.0.0.1 - - [16/Sep/2024 21:46:22] "OPTIONS /qahandbook/remote%20word HTTP/1.1" 200 -
127.0.0.1 - - [16/Sep/2024 21:46:22] "POST /qahandbook/remote%20word HTTP/1.1" 200 -
127.0.0.1 - - [16/Sep/2024 21:46:30] "OPTIONS /qahandbook/remote HTTP/1.1" 200 -
127.0.0.1 - - [16/Sep/2024 21:46:30] "POST /qahandbook/remote HTTP/1.1" 200 -
127.0.0.1 - - [16/Sep/2024 21:46:35] "OPTIONS /qahandbook/cavation HTTP/1.1" 200 -
127.0.0.1 - - [16/Sep/2024 21:46:35] "POST /qahandbook/cavation HTTP/1.1" 200 -
